In [8]:
import asyncio
import os
import asyncio

from typing import Annotated
from openai import AsyncOpenAI


from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory


from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

from semantic_kernel.connectors.ai import FunctionChoiceBehavior

from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

In [3]:
# Define a sample plugin for the sample
class DestinationsPlugin:
    """A List of Destinations for vacation."""

    @kernel_function(description="Provides a list of vacation desinations.")
    def get_destinations(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Barcelona, Spain
        Paris, France
        Berlin, Germany
        """

    @kernel_function(description="Provides the availability of a destination.")
    def get_availability(
        self, destination: Annotated[str, "The destination to check availability for."]
    ) -> Annotated[str, "Returns the availability of the destination."]:
        return """
        Barcelona - Unavailable
        Paris - Available
        Berlin - Available
        """

In [6]:
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")

kernel = Kernel()
kernel.add_plugin(DestinationsPlugin(), plugin_name="destinations")

service_id = "agent"

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
    service_id=service_id
)
kernel.add_service(chat_completion_service)

In [9]:
settings = kernel.get_prompt_execution_settings_from_service_id(
    service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

In [10]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = "Answer questions about the travel destinations and their availability."
# Create the agent
agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings),
)

In [12]:
async def main():
    # Define the chat history
    chat_history = ChatHistory()

    # Respond to user input
    user_inputs = [
        "What destinations are available?",
        "Is Barcelona available?",
    ]

    for user_input in user_inputs:
        # Add the user input to the chat history
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")

        agent_name: str | None = None
        print("# Assistant - ", end="")
        async for content in agent.invoke_stream(chat_history):
            if not agent_name:
                agent_name = content.name
                print(f"{agent_name}: '", end="")
            if (
                not any(isinstance(item, (FunctionCallContent, FunctionResultContent))
                        for item in content.items)
                and content.content.strip()
            ):
                print(f"{content.content}", end="", flush=True)
        print("'")

await main()

# User: 'What destinations are available?'
# Assistant - TravelAgent: 'The available travel destinations are:
- Barcelona, Spain- Paris, France- Berlin, Germany'
# User: 'Is Barcelona available?'
# Assistant - TravelAgent: 'Barcelona is currently unavailable. However, Paris and Berlin are available.'
